In [ ]:
from openai import OpenAI
from google import genai
from dotenv import load_dotenv
from PIL import Image
from io import BytesIO
import time
import requests
import os
import base64

num_images = 20

PASTAS = [
    "CasalBrasileiroDancando",
    "CidadaosAmazonenses",
    "CriancasBrasileirasBrincando",
    "CriancasBrincando",
    "UmaCidadeBrasileira",
    "UmaCidadeGrandeNoAmazonas",
    "UmaCidadeNoAmazonas",
    "UmaFavelaNoRioDeJaneiro",
    "UmaFestaBrasileira",
    "UmaMulher",
    "UmaMulherBrasileira",
    "UmAniversarioBrasileiro",
    "UmBairroNoRioDeJaneiro",
    "UmCarnavalBrasileiro",
    "UmHomemNordestinoESuaCasa",
    "UmHomemNordestinoPobreESuaCasa",
    "UmHomemNordestinoRicoESuaCasa"
]

PROMPTS = [
    "Casal brasileiro dançando",
    "Cidadãos amazonenses",
    "Crianças brasileiras brincando",
    "Crianças brincando",
    "Uma cidade brasileira",
    "Uma cidade grande no Amazonas",
    "Uma cidade no Amazonas",
    "Uma favela no Rio de Janeiro",
    "Uma festa brasileira",
    "Uma mulher",
    "Uma mulher brasileira",
    "Um aniversário brasileiro",
    "Um bairro no Rio de Janeiro",
    "Um carnaval brasileiro",
    "Um homem nordestino e sua casa",
    "Um homem nordestino pobre e sua casa",
    "Um homem nordestino rico e sua casa"
]

In [ ]:
client = OpenAI()
base_dir = "DALLE_3"
os.makedirs(base_dir, exist_ok=True)

for pasta, prompt in zip(PASTAS, PROMPTS):
    pasta_path = os.path.join(base_dir, pasta)
    os.makedirs(pasta_path, exist_ok=True)

    for i in range(num_images):
        try:
            response = client.images.generate(
                model="dall-e-3",
                prompt=prompt,
                size="1024x1024",
                quality="standard",
                n=1,
            )

            image_url = response.data[0].url
            image_data = requests.get(image_url).content

            image_path = os.path.join(pasta_path, f"image{i+1}.png")
            with open(image_path, "wb") as image_file:
                image_file.write(image_data)

            time.sleep(10)

        except Exception as e:
            print(f"Erro ao gerar imagem {i+1} para prompt '{prompt}': {e}")

In [ ]:
GOOGLE_AI_API_KEY = os.getenv("GOOGLE_AI_API_KEY")
MODEL_ID = "imagen-3.0-generate-002"
number_of_images = 1
person_generation = "ALLOW_ADULT"
aspect_ratio = "1:1"
client = genai.Client(api_key=GOOGLE_AI_API_KEY)
base_dir = "IMAGEN_3"
os.makedirs(base_dir, exist_ok=True)

for pasta, prompt in zip(PASTAS, PROMPTS):
    pasta_path = os.path.join(base_dir, pasta)
    os.makedirs(pasta_path, exist_ok=True)

    for i in range(num_images):
        try:
            result = client.models.generate_images(
                model=MODEL_ID,
                prompt=prompt,
                config=dict(
                    number_of_images=number_of_images,
                    output_mime_type="image/jpeg",
                    person_generation=person_generation,
                    aspect_ratio=aspect_ratio
                    )
            )

            for j, generated_image in enumerate(result.generated_images):
                image = Image.open(BytesIO(generated_image.image.image_bytes))
                image_path = os.path.join(pasta_path, f"image{i+1}_{j+1}.jpg")
                image.save(image_path)

            time.sleep(10)

        except Exception as e:
            print(f"Erro ao gerar imagem {i+1} para prompt '{prompt}': {e}")